# Tracking Amateur Radio Satellites using Skyfield

By Jari Perkiömäki OH6BG

In this example, we calculate the times of visibility of amateur radio satellites, i.e. when the satellites are above the horizon in the Observer's geographical location. Typically, satellites can cross over the Observer's location quite a number of times each day. Also, we calculate the azimuth and the distance from the Observer to the satellite.

The resolution for calculations is one minute.

We are using the new Skyfield module from https://github.com/skyfielders/python-skyfield by Brandon Rhodes. This script requires an Internet connection as the TLE (Two-Line Element set) satellite data need to be downloaded.

Install the Skyfield module as follows:

In [ ]:
pip install skyfield

This is a plain vanilla Python 3 code to show the skeleton for such calculations. Unfortunately, it will run a little bit slowly. This is a known issue but I did not try to optimize the code to make it run faster.

Let's first load the Skyfield module.

In [ ]:
from skyfield.api import Topos, load

ts = load.timescale()
t = ts.now()

We will need to know the Observer's geographical coordinates, and the date we are interested in.

In [ ]:
# Observer coordinates
# lat = decimal latitude, positive for N, negative for S
# lon = decimal longitude, positive for E, negative for W
lat, lon = 40.7128, -74.0060  # New York

# Observer date
year, month, day = 2018, 12, 17

Let's initialize some variables and the output filename, and define the URL where the satellite TLE (two-line element set, see https://en.wikipedia.org/wiki/Two-line_element_set for clarification) data can be loaded. Finally, we will create the Skyfield Topos object from the Observer's coordinates.

In [ ]:
# initialize some parameters
# i = a variable to count the number of unique satellites in the "amateur.txt" file
# refreshed = a Boolean variable used when we will need to reload TLE data
# sat_list = a list of unique satellite names
i, refreshed, sat_list = 0, False, []

# sat_file = the name of the output file, together with the date
sat_file = "amateur_satellite_times_%s-%s-%s.txt" % (year, month, day)

# the URL for the TLE satellite data
stations_url = 'https://www.celestrak.com/NORAD/elements/amateur.txt'

# load.tle() takes care of fetching the satellite TLE data from the Internet and parsing it
satellites = load.tle(stations_url)

# the Topos object describes the Observer's geographic location
qth = Topos(lat, lon)

In [ ]:
# some response on the screen to the user as the program starts to calculate the satellite data
res = "AMATEUR SATELLITE TIMES FOR LOCATION: %s, %s" % (lat, lon)
print(res)
print("Date:", "-".join([str(year), str(month), str(day)]), "\n")

Now, we loop the list of satellites, defined in NORAD's "amateur.txt" file. Unfortunately, we cannot just enumerate all the satellites as each satellite is parsed two to four times. Consequently, first, we will need to make sure that we only process unique satellites.

In [ ]:
for sat in satellites:

    # if we encounter a satellite only identified by a number, we will skip it
    if isinstance(sat, int):
        continue

    satellite = satellites[sat]
    
    # sid or satellite ID is a unique number for each satellite
    sid = str(satellite).rsplit(' ', 2)[1].split("=")[1]

    # we are keeping a list of all unique satellite IDs and, if already in the list, we will not process it
    if sid not in sat_list:
        sat_list.append(sid)
    else:
        continue

We will now check that the TLE satellite data is as fresh as possible to give us as accurate results as possible. If the TLE data is more than two days old, and has not refreshed earlier (note this check is done in a loop for all satellites), the script tries to reload all the necessary data, and continue.

In [ ]:
    difference = satellite - qth
    days = t - satellite.epoch

    if abs(days) > 2 and not refreshed:
        satellites = load.tle(stations_url, reload=True)
        refreshed = True
        satellite = satellites[sat]
        difference = satellite - qth
        days = t - satellite.epoch

We will initialize the unique satellite counter ("i"), and print some data on the screen for the user to show the progress of the script. Also, we will use the in-memory variable "res" to write the actual results to. The content of the variable "res" will finally be written to a file at the end of this script.

In [ ]:
    i += 1
    print("Processing [%02d] %s: %.3f days away from epoch" % (i, sat, days))
    res += "\n%s: %.3f days away from epoch\n" % (sat, days)
    res += "%-16s  %3s  %-4s  %5s\n" % ("DATETIME",
        "ALT",
        "AZIM",
        "KM",)

For each satellite, we will calculate all 24 hours (hh) and, within all hours, all 60 minutes. The Boolean variable "sep" determines if we will later need to print a separator line in the results to distinguish the runs from each other.

In [ ]:
    sep = True

    for hh in range(24):

        for mm in range(60):

            t = ts.utc(year, month, day, hh, mm)
            topocentric = difference.at(t)
            alt, az, distance = topocentric.altaz()

If the altitude (the variable "alt"), or height above the horizon in degrees, is less than -0.5 degrees, then we will set the "sep" variable to True again to wait for the next instance where the satellite is visible again for the Observer. If so, we will need to print the separator line.

When printing the results to the (output) variable "sep", note that we are using the UTC (Coordinated Universal Time) and not the Observer's local time. We are also formatting the datetime string as follows: <b>%Y-%m-%d %H:%M</b> [year-month-day hours:minutes], e.g. "2018-12-17 21:29".

In [ ]:
            if alt.degrees < -0.5:
                sep = True
            else:
                if sep:
                    res += "%s\n" % ("-" * 34)
                    sep = False

                res += "%s  %2d°  %3d°  %5d\n" % (t.utc_strftime('%Y-%m-%d %H:%M'),
                    round(alt.degrees),
                    round(az.degrees),
                    round(distance.km),)

All calculations are now ready, and the results will be written to 'sat_file'. Finally, we are printing a message, noting that the output file has been written to disk.

In [ ]:
with open(sat_file, 'w', encoding='utf-8') as outfile:
    outfile.write(res)

print("Writing satellite times to: %s" % sat_file)